# Noise Field

In this notebook we want to compute the distance field of the busy street the Heer Bokelweg which is approximately 100 meters away from the plot. This street will most likely produce noise for our building. This is why we would like to know how much noise there will be.

## 0. Initialization

### 0.1. Load required libraries

In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import scipy as sp

### 0.2. Load the envelope lattice as the avialbility lattice

In [2]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/final_envelope_new.csv')
avail_lattice = tg.lattice_from_csv(lattice_path)
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice), avail_lattice)

### 0.3. Load noise sources

In [3]:
# loading program (agents information) from CSV
noise_source_path = os.path.relpath('../data/noise_points_new.csv')
noise_sources = np.genfromtxt(noise_source_path, delimiter=',')

## 1. Creation of Noise Field

### 1.1. Computing noise lattices

In [4]:
# create full lattice
full_lattice = avail_lattice * 0 + 1

# extract the coordiantes of the centroid of all voxel
vox_centroids = full_lattice.centroids

# extract voxel indices of all voxels
vox_indices = np.array(np.where(full_lattice==1)).T

# setting the noise base pressure level 
# this level we got from the noise map of the "Atlas van de Leefomgeving"
noise_base = 70.0

# initializing the sum lattice of noise
sum_noise_lats = avail_lattice * 0.0

# for each source of noise
for noise_src in noise_sources:
    # initialize the occupation lattice
    dist_latice = avail_lattice * 0.0

    for cen, ind in zip(vox_centroids, vox_indices):
        # compute the euclidian distance
        dist_latice[tuple(ind)] = sp.spatial.distance.euclidean(cen, noise_src)

    # computing the noise lattice from dist lattice
    noise_latice = noise_base - 10 * np.log10(dist_latice) - 5
    
    # summing
    sum_noise_lats += np.power(10, noise_latice / 10.0)


# computing the final aggregation
agg_noise_lats = 10 * np.log10(sum_noise_lats)

### 1.2. Visualizing the noise lattices

In [5]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# load the mesh from file
context_path = os.path.relpath('../data/extended_context.obj')
context_mesh = tm.load(context_path)

# Create the spatial reference
grid = pv.UniformGrid()

vis_lattice = agg_noise_lats

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = vis_lattice.shape
# The bottom left corner of the data set
grid.origin = vis_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = vis_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Noise"] = vis_lattice.flatten(order="F")  # Flatten the Lattice

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), opacity=1.0, style='surface')

# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])*1.5
p.add_volume(grid, cmap="coolwarm" ,opacity=opacity, shade=True)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(1437.4594008982308, 1436.248814398231, 1449.288430048231),
 (18.23971549999999, 17.02912900000001, 30.06874465),
 (0.0, 0.0, 1.0)]

### 1.3. Save Noise lattice to CSV

In [6]:
# save the sun access latice to csv

csv_path = os.path.relpath('../data/HeerBokelweg_noise.csv')
agg_noise_lats.to_csv(csv_path)

### Credits

In [7]:
__author__ = "Shervin Azadi"
__changes_made_by__ = "Frank Vahstal"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/frankvahstal/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Noise Fields"